https://blog.csdn.net/sinat_28442665/article/details/84029633
Show how to check the information about WAV
https://blog.csdn.net/qq_38563206/article/details/126486544
Show how to Mixing, overlaying audio, splicing audio, here we need to do overlaying audio.

So they use FSD50K, ESC-50, MUSDB18, DISCO
https://research.google.com/audioset/download.html

Starting from 2 sounds

In [12]:
# read data from FSD50K
datasetpath = '../data/BinauralCuratedDataset/FSD50K/'
ds_dev_path = datasetpath + 'FSD50K.dev_audio/'
ds_dev_gt_path = datasetpath + 'FSD50K.ground_truth/dev.csv'

In [13]:
# read labels grom gt, and sort
# import block
import pandas as pd
import os
from tqdm import tqdm
import numpy as np


In [14]:
# get labels and seperate into single labels.
# print(os.path.exists('../data/BinauralCuratedDataset/FSD50K/FSD50K.ground_truth/dev.csv'))
df_dev_gt = pd.read_csv(ds_dev_gt_path)
df_dev_gt = df_dev_gt.get(['fname', 'labels'])
df_dev_gt_sep = pd.DataFrame(columns=['fname', 'label'])
for i, v in tqdm(enumerate(df_dev_gt.index)):
    t1 = df_dev_gt['labels'][i].split(',')
    t2 = df_dev_gt['fname'][i]
    l = len(t1)
    for i2, v2 in enumerate(t1):
        df_dev_gt_sep = pd.concat([df_dev_gt_sep, pd.DataFrame([[t2,v2]], columns=df_dev_gt_sep.columns)], ignore_index=True)
df_dev_gt.shape
# this may have a problem because some classes are too closed, for example keyboard and piano. It is a muti-label prediction. 

40966it [00:48, 837.68it/s] 


(40966, 2)

In [15]:
print(df_dev_gt.shape)
print(df_dev_gt_sep.shape)

(40966, 2)
(114271, 2)


In [49]:
# create a two sound mixture.
# select two data from the df_dev_gt data, where they dont share any same label. 
# if they have multiple labels, we choose a random one.
def check_share_attribute(s1, s2):
    l1 = s1.split(',')
    l2 = s2.split(',')
    if list(set(l1)&set(l2)):
        return True
    else:
        return False
np.random.seed(66)
def return_drop_rate(rate):
    return np.random.random() > rate

df_dev_mix_no_share = pd.DataFrame(columns=['newfname', 'fname1', 'fname2', 'labels', 'gt_label', 'gt_index'])
# let's make a some mixtures first
for i, v in tqdm(enumerate(df_dev_gt.index)):
    if not return_drop_rate(0.99):
            continue
    for i2, v2 in enumerate(df_dev_gt.index):
        if not return_drop_rate(0.99):
            continue
        if i >= i2:
            continue
        l1 = df_dev_gt['labels'][i]
        l2 = df_dev_gt['labels'][i2]
        r1 = np.random.randint(0, len(l1.split(',')))
        r2 = np.random.randint(0, len(l2.split(',')))
        l1 = l1.split(',')[r1]
        l2 = l2.split(',')[r2]
        r3 = np.random.randint(0, 2)
        if check_share_attribute(df_dev_gt['labels'][i], df_dev_gt['labels'][i2]):
            continue
        else:
            newrow = [str(df_dev_gt['fname'][i])+'_'+str(df_dev_gt['fname'][i2]), str(df_dev_gt['fname'][i]), str(df_dev_gt['fname'][i2]), l1+','+l2, [l1,l2][r3], r3]
            df_dev_mix_no_share = pd.concat([df_dev_mix_no_share, pd.DataFrame([newrow], columns=df_dev_mix_no_share.columns)], ignore_index=True)
df_dev_mix_no_share.shape
        
        

40966it [01:30, 453.92it/s] 


(73198, 6)

In [50]:
#randomly use 10k data for small experiement:
small_box_experiment = np.random.choice(df_dev_mix_no_share.index, 10000, replace=False)
df_dev_mix_no_share_small = df_dev_mix_no_share.loc[small_box_experiment]

small_box_test=df_dev_mix_no_share_small.sample(frac=0.2)
t_df=df_dev_mix_no_share_small[~df_dev_mix_no_share_small.index.isin(small_box_test.index)]
small_box_val=t_df.sample(frac=0.25)
small_box_train=t_df[~t_df.index.isin(small_box_val.index)]

In [60]:
# make dataset for the small experiment
from pydub import AudioSegment
import librosa

wav_path = '../data/BinauralCuratedDataset/FSD50K/FSD50K.dev_audio/'
for i, wav_t in tqdm(small_box_train.iterrows()):
    sound1 = AudioSegment.from_wav(wav_path+wav_t['fname1']+'.wav')
    sound2 = AudioSegment.from_wav(wav_path+wav_t['fname2']+'.wav')
    if len(sound1) > len(sound2):
        output = sound1.overlay(sound2) # overlay them
    else:
        output = sound2.overlay(sound1)
    output.export('./data_folder/train/'+wav_t['newfname']+'.wav', format="wav") 



6000it [00:07, 802.98it/s]


In [61]:
# make dataset for the small experiment
from pydub import AudioSegment
import librosa

wav_path = '../data/BinauralCuratedDataset/FSD50K/FSD50K.dev_audio/'
for i, wav_t in tqdm(small_box_val.iterrows()):
    sound1 = AudioSegment.from_wav(wav_path+wav_t['fname1']+'.wav')
    sound2 = AudioSegment.from_wav(wav_path+wav_t['fname2']+'.wav')
    if len(sound1) > len(sound2):
        output = sound1.overlay(sound2) # overlay them
    else:
        output = sound2.overlay(sound1)
    output.export('./data_folder/val/'+wav_t['newfname']+'.wav', format="wav") 



2000it [00:04, 419.14it/s]


In [62]:
# make dataset for the small experiment
from pydub import AudioSegment
import librosa

wav_path = '../data/BinauralCuratedDataset/FSD50K/FSD50K.dev_audio/'
for i, wav_t in tqdm(small_box_test.iterrows()):
    sound1 = AudioSegment.from_wav(wav_path+wav_t['fname1']+'.wav')
    sound2 = AudioSegment.from_wav(wav_path+wav_t['fname2']+'.wav')
    if len(sound1) > len(sound2):
        output = sound1.overlay(sound2) # overlay them
    else:
        output = sound2.overlay(sound1)
    output.export('./data_folder/test/'+wav_t['newfname']+'.wav', format="wav") 



2000it [00:07, 274.96it/s]


In [ ]:
# store 
df_dev_mix_no_share_small.to_csv('data_folder/df_dev_mix_no_share_small.csv', index=False)  